## 006 Model comparison for RL models (all subjects)

Date: 23.Jul.2022

### Purpose

Fit RW and non-RL models to all participants and perform model comparison.

### Outline of Plan

1. Loop over all subjects in longform to fit RL and non-RL models.
2. For each subject trace, calculate the WAIC/LOO model comparison.
3. For each model, store the distance to the best performing model.
4. Across subjects plot the WAIC distance to best model for each model.

### What we did in this notebook

1. Copied this over to exploration folder for food craving, better suited there.


In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import pymc as pm
import aesara.tensor as at
import aesara

from sys import path

root_dir = "/Users/kulkarnik/LocalProjects/SlotsTasks/"
project_dir = f'{root_dir}/online/prolific-food-craving/'
model_functions_path = f'{root_dir}/bayesian_models/slotscraving/'

## Add model_functions to system path
path.append(model_functions_path)

from sepblock_decision.utils import load_data # type: ignore

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
import arviz as az

## Load data

In [4]:
path_to_summary = f'{project_dir}/rawdata/clean_df_summary.csv'
path_to_longform = f'{project_dir}/rawdata/clean_df_longform.csv'
df_summary, longform = load_data.load_clean_dbs(path_to_summary, path_to_longform)
df_summary = df_summary[
    (df_summary['Money Accuracy']>0.4) & 
    (df_summary['Other Accuracy']>0.4)]
longform = longform[(longform['pid_db'].isin(df_summary['id'])) & (longform['Type']!='practice')]
pid_list = longform['PID'].unique()
longform

,PID,id,Block,Type,Trial,Cue Time,Action,Action Time,Reward,Reward Time,RT,Spin Speed,Craving Rating,Mood Rating,pid_db
5,61281debe85082cc937dd9ae,6,1,other,1,1.647963e+09,1,1.647963e+09,1,1.647963e+09,0.972,fast,-1.0,-1.0,1
6,61281debe85082cc937dd9ae,7,1,other,2,1.647963e+09,1,1.647963e+09,0,1.647963e+09,0.045,fast,35.0,-1.0,1
7,61281debe85082cc937dd9ae,8,1,other,3,1.647963e+09,0,1.647963e+09,1,1.647963e+09,0.718,slow,-1.0,-1.0,1
8,61281debe85082cc937dd9ae,9,1,other,4,1.647963e+09,0,1.647963e+09,0,1.647963e+09,0.030,slow,-1.0,-1.0,1
9,61281debe85082cc937dd9ae,10,1,other,5,1.647963e+09,0,1.647963e+09,0,1.647963e+09,0.500,fast,34.0,18.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5620,6102d5e6ff92a8dd0cce6b8f,5700,2,other,56,1.648471e+09,1,1.648471e+09,1,1.648471e+09,0.754,slow,-1.0,-1.0,47
5621,6102d5e6ff92a8dd0cce6b8f,5701,2,other,57,1.648471e+09,1,1.648471e+09,1,1.648471e+09,0.637,slow,-1.0,-1.0,47
5622,6102d5e6ff92a8dd0cce6b8f,5702,2,other,58,1.648471e+09,1,1.648471e+09,1,1.648471e+09,0.505,fast,-1.0,-1.0,47
5623,6102d5e6ff92a8dd0cce6b8f,5703,2,other,59,1.648471e+09,1,1.648471e+09,1,1.648471e+09,0.652,fast,-1.0,-1.0,47


## Model classes

In [15]:
class Biased:
    def __init__(self, longform, summary):
        self.name = 'Biased'
        self.longform = longform
        self.summary = summary
        self.pid_list = longform['PID'].unique()
        self.traces = {
            'money': {},
            'other': {}
        }

    def right_action_probs(self, sample_bias, actions, rewards):
        t_rewards = at.as_tensor_variable(rewards, dtype='int32')
        t_actions = at.as_tensor_variable(actions, dtype='int32')

        # Return the probabilities for the right action, in the original scale
        return at.repeat(sample_bias, t_actions.shape[1])

    def fit(self, pid_num, block):
        pid = self.pid_list[pid_num]
        act = self.longform[(self.longform['PID']==pid) & (self.longform['Type']==block)]['Action'].values
        rew = self.longform[(self.longform['PID']==pid) & (self.longform['Type']==block)]['Reward'].values
        with pm.Model() as model:
            untr_bias = pm.Normal('untr_bias', mu=0, sigma=1)
            bias = pm.Deterministic('bias', pm.math.invlogit(untr_bias))

            # action_probs = self.right_action_probs(bias, act, rew)
            like = pm.Bernoulli('like', p=bias, observed=act[1:])

            self.trace[pid][block] = pm.sample()
            self.trace[pid][block].extend(pm.sample_prior_predictive())
            pm.sample_posterior_predictive(self.trace[pid][block], extend_inferencedata=True)

In [8]:
class Heuristic:
    def __init__(self, longform, summary):
        self.name = 'Heuristic'
        self.longform = longform
        self.summary = summary
        self.pid_list = longform['PID'].unique()
        self.trace = {}
    
    def add_eps(self, st, a, eps_t, eps):
        return aesara.ifelse.ifelse(
                at.eq(st, 1),
                aesara.ifelse.ifelse(
                    at.eq(a, 1),
                    eps,
                    1-eps
                ),
                aesara.ifelse.ifelse(
                    at.eq(a, 1),
                    1-eps,
                    eps
                )
            )

    def right_action_probs(self, sample_eps, actions, strat):
        t_strat = at.as_tensor_variable(strat, dtype='int32')
        t_actions = at.as_tensor_variable(actions, dtype='int32')

        # Compute the Qs values
        t_eps = at.as_tensor_variable(np.asarray(1, 'float64'))
        t_eps, updates = aesara.scan(
            fn=self.add_eps,
            sequences=[t_strat, t_actions],
            outputs_info=t_eps,
            non_sequences=[sample_eps])
        return t_eps

    def fit(self, pid_num, block):
        pid = self.pid_list[pid_num]
        act = self.longform[(self.longform['PID']==pid) & (self.longform['Type']==block)]['Action'].values
        rew = self.longform[(self.longform['PID']==pid) & (self.longform['Type']==block)]['Reward'].values
        strat = np.zeros(len(act))
        for i, a in enumerate(act):
            if i < 2:
                continue
            should_switch = np.all(np.array([rew[i-2]==rew[i-1], rew[i-1]==0]), axis=0)
            do_switch = act[i-1]!=act[i]
            strat[i] = should_switch==do_switch
        strat = strat.astype(int)
        
        with pm.Model() as model:
            untr_eps = pm.Normal('untr_eps', mu=0, sigma=1)
            eps = pm.Deterministic('eps', pm.math.invlogit(untr_eps))

            action_probs = self.right_action_probs(eps, strat, act)
            like = pm.Bernoulli('like', p=action_probs[1:], observed=act[1:])

            self.trace[pid] = pm.sample()
            self.trace[pid].extend(pm.sample_prior_predictive())
            pm.sample_posterior_predictive(self.trace[pid], extend_inferencedata=True)

In [9]:
class RW:
    def __init__(self, longform, summary):
        self.name = 'RW'
        self.longform = longform
        self.summary = summary
        self.pid_list = longform['PID'].unique()
        self.trace = {}

    def update_Q(self, a, r, Qs, al):
        Qs = at.set_subtensor(Qs[a], Qs[a] + al * (r - Qs[a]))
        return Qs

    def right_action_probs(self, sample_alpha, sample_beta, actions, rewards):
        t_rewards = at.as_tensor_variable(rewards, dtype='int32')
        t_actions = at.as_tensor_variable(actions, dtype='int32')

        # Compute the Qs values
        t_Qs = 0.5 * at.ones((2,), dtype='float64')
        t_Qs, updates = aesara.scan(
            fn=self.update_Q,
            sequences=[t_actions, t_rewards],
            outputs_info=[t_Qs],
            non_sequences=[sample_alpha])

        # Apply the sotfmax transformation
        t_Qs = t_Qs[:-1] * sample_beta
        logp_actions = t_Qs - at.logsumexp(t_Qs, axis=1, keepdims=True)

        # Return the probabilities for the right action, in the original scale
        return at.exp(logp_actions[:, 1]) 

    def fit(self, pid_num, block):
        pid = self.pid_list[pid_num]
        act = self.longform[(self.longform['PID']==pid) & (self.longform['Type']==block)]['Action'].values
        rew = self.longform[(self.longform['PID']==pid) & (self.longform['Type']==block)]['Reward'].values
        with pm.Model() as model:
            untr_alpha = pm.Normal('untr_alpha', mu=0, sigma=1)
            beta = pm.HalfNormal('beta', 10)
            alpha = pm.Deterministic('alpha', pm.math.invlogit(untr_alpha))

            action_probs = self.right_action_probs(alpha, beta, act, rew)
            like = pm.Bernoulli('like', p=action_probs, observed=act[1:])

            self.trace[pid] = pm.sample()
            self.trace[pid].extend(pm.sample_prior_predictive())
            pm.sample_posterior_predictive(self.trace[pid], extend_inferencedata=True)

In [10]:
class RWDecay:
    def __init__(self, longform, summary):
        self.name = 'RWDecay'
        self.longform = longform
        self.summary = summary
        self.pid_list = longform['PID'].unique()
        self.trace = {}

    def update_Q(self, a, r, Qs, al, d):
        Qs = at.set_subtensor(Qs[a], Qs[a] + al * (r - Qs[a]))
        Qs = at.set_subtensor(Qs[1-a], Qs[1-a] + d * (0.5 - Qs[1-a]))
        return Qs

    def right_action_probs(self, sample_alpha, sample_beta, sample_decay, actions, rewards):
        t_rewards = at.as_tensor_variable(rewards, dtype='int32')
        t_actions = at.as_tensor_variable(actions, dtype='int32')

        # Compute the Qs values
        t_Qs = 0.5 * at.ones((2,), dtype='float64')
        t_Qs, updates = aesara.scan(
            fn=self.update_Q,
            sequences=[t_actions, t_rewards],
            outputs_info=[t_Qs],
            non_sequences=[sample_alpha, sample_decay])

        # Apply the sotfmax transformation
        t_Qs = t_Qs[:-1] * sample_beta
        logp_actions = t_Qs - at.logsumexp(t_Qs, axis=1, keepdims=True)

        # Return the probabilities for the right action, in the original scale
        return at.exp(logp_actions[:, 1]) 

    def fit(self, pid_num, block):
        pid = self.pid_list[pid_num]
        act = self.longform[(self.longform['PID']==pid) & (self.longform['Type']==block)]['Action'].values
        rew = self.longform[(self.longform['PID']==pid) & (self.longform['Type']==block)]['Reward'].values
        with pm.Model() as model:
            untr_alpha = pm.Normal('untr_alpha', mu=0, sigma=1)
            untr_decay = pm.Normal('untr_decay', mu=0, sigma=1)
            beta = pm.HalfNormal('beta', 10)
            alpha = pm.Deterministic('alpha', pm.math.invlogit(untr_alpha))
            decay = pm.Deterministic('decay', pm.math.invlogit(untr_decay))

            action_probs = self.right_action_probs(alpha, beta, decay, act, rew)
            like = pm.Bernoulli('like', p=action_probs, observed=act[1:])

            self.trace[pid] = pm.sample()
            self.trace[pid].extend(pm.sample_prior_predictive())
            pm.sample_posterior_predictive(self.trace[pid], extend_inferencedata=True)

In [11]:
class RWFictive:
    def __init__(self, longform, summary):
        self.name = 'RWFictive'
        self.longform = longform
        self.summary = summary
        self.pid_list = longform['PID'].unique()
        self.trace = {}

    def update_Q(self, a, r, Qs, al, f_al):
        Qs = at.set_subtensor(Qs[a], Qs[a] + al * (r - Qs[a]))
        Qs = at.set_subtensor(Qs[1-a], Qs[1-a] + f_al * (r - Qs[a]))
        return Qs

    def right_action_probs(self, sample_alpha, sample_fictive_alpha, sample_beta, actions, rewards):
        t_rewards = at.as_tensor_variable(rewards, dtype='int32')
        t_actions = at.as_tensor_variable(actions, dtype='int32')

        # Compute the Qs values
        t_Qs = 0.5 * at.ones((2,), dtype='float64')
        t_Qs, updates = aesara.scan(
            fn=self.update_Q,
            sequences=[t_actions, t_rewards],
            outputs_info=[t_Qs],
            non_sequences=[sample_alpha, sample_fictive_alpha])

        # Apply the sotfmax transformation
        t_Qs = t_Qs[:-1] * sample_beta
        logp_actions = t_Qs - at.logsumexp(t_Qs, axis=1, keepdims=True)

        # Return the probabilities for the right action, in the original scale
        return at.exp(logp_actions[:, 1]) 

    def fit(self, pid_num, block):
        pid = self.pid_list[pid_num]
        act = self.longform[(self.longform['PID']==pid) & (self.longform['Type']==block)]['Action'].values
        rew = self.longform[(self.longform['PID']==pid) & (self.longform['Type']==block)]['Reward'].values
        with pm.Model() as model:
            untr_alpha = pm.Normal('untr_alpha', mu=0, sigma=1)
            untr_fictive_alpha = pm.Normal('untr_fictive_alpha', mu=0, sigma=1)
            beta = pm.HalfNormal('beta', 10)
            alpha = pm.Deterministic('alpha', pm.math.invlogit(untr_alpha))
            fictive_alpha = pm.Deterministic('fictive_alpha', pm.math.invlogit(untr_fictive_alpha))

            action_probs = self.right_action_probs(alpha, fictive_alpha, beta, act, rew)
            like = pm.Bernoulli('like', p=action_probs, observed=act[1:])

            self.trace[pid] = pm.sample()
            self.trace[pid].extend(pm.sample_prior_predictive())
            pm.sample_posterior_predictive(self.trace[pid], extend_inferencedata=True)

In [12]:
class RWSeparate:
    def __init__(self, longform, summary):
        self.name = 'RWSeparate'
        self.longform = longform
        self.summary = summary
        self.pid_list = longform['PID'].unique()
        self.trace = {}

    def update_Q(self, a, r, Qs, pos_al, neg_al):
        Qs = aesara.ifelse.ifelse(
            at.lt(r-Qs[a], 0),
            at.set_subtensor(Qs[a], Qs[a] + neg_al * (r - Qs[a])),
            at.set_subtensor(Qs[a], Qs[a] + pos_al * (r - Qs[a]))
        )
        
        return Qs

    def right_action_probs(self, sample_pos_alpha, sample_neg_alpha, sample_beta, actions, rewards):
        t_rewards = at.as_tensor_variable(rewards, dtype='int32')
        t_actions = at.as_tensor_variable(actions, dtype='int32')

        # Compute the Qs values
        t_Qs = 0.5 * at.ones((2,), dtype='float64')
        t_Qs, updates = aesara.scan(
            fn=self.update_Q,
            sequences=[t_actions, t_rewards],
            outputs_info=[t_Qs],
            non_sequences=[sample_pos_alpha, sample_neg_alpha])

        # Apply the sotfmax transformation
        t_Qs = t_Qs[:-1] * sample_beta
        logp_actions = t_Qs - at.logsumexp(t_Qs, axis=1, keepdims=True)

        # Return the probabilities for the right action, in the original scale
        return at.exp(logp_actions[:, 1]) 

    def fit(self, pid_num, block):
        pid = self.pid_list[pid_num]
        act = self.longform[(self.longform['PID']==pid) & (self.longform['Type']==block)]['Action'].values
        rew = self.longform[(self.longform['PID']==pid) & (self.longform['Type']==block)]['Reward'].values
        with pm.Model() as model:
            untr_alpha = pm.Normal('untr_pos_alpha', mu=0, sigma=1)
            untr_fictive_alpha = pm.Normal('untr_neg_alpha', mu=0, sigma=1)
            beta = pm.HalfNormal('beta', 10)
            pos_alpha = pm.Deterministic('pos_alpha', pm.math.invlogit(untr_alpha))
            neg_alpha = pm.Deterministic('neg_alpha', pm.math.invlogit(untr_fictive_alpha))

            action_probs = self.right_action_probs(pos_alpha, neg_alpha, beta, act, rew)
            like = pm.Bernoulli('like', p=action_probs, observed=act[1:])

            self.trace[pid] = pm.sample()
            self.trace[pid].extend(pm.sample_prior_predictive())
            pm.sample_posterior_predictive(self.trace[pid], extend_inferencedata=True)

## Loop over subjects for each model

In [13]:
biased_model = Biased(longform, df_summary)
heu_model = Heuristic(longform, df_summary)
rw_model = RW(longform, df_summary)
rwdecay_model = RWDecay(longform, df_summary)
rwseparate_model = RWSeparate(longform, df_summary)
rwfictive_model = RWFictive(longform, df_summary)

In [14]:
df_summary.shape

(43, 10)

In [16]:
for i in np.arange(df_summary.shape[0]):
    for bl in ['money', 'other']:
        biased_model.fit(i, bl)
    break


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [untr_bias]


/Users/kulkarnik/miniconda3/envs/jupyter/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kulkarnik/miniconda3/envs/jupyter/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kulkarnik/miniconda3/envs/jupyter/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kulkarnik/miniconda3/envs/jupyter/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 22 seconds.


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [untr_bias]


/Users/kulkarnik/miniconda3/envs/jupyter/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kulkarnik/miniconda3/envs/jupyter/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kulkarnik/miniconda3/envs/jupyter/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 20 seconds.
